In [2]:
# text from https://www.reedbeta.com/blog/programmers-intro-to-unicode/
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
text_bytes = text.encode("utf-8") # raw bytes
text_byteints = list(map(int, text_bytes)) # convert to a list of integers in range 0..255 for convenience
print('---')
print(text)
print("length:", len(text))
print('---')
print(text_byteints)
print("length:", len(text_byteints))

---
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
length: 533
---
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140

In [ ]:
# character lengths 533
# byte integers length 616
# Byte-BPE to compress

In [ ]:
# BPE 的基本思想就是 occur-most pair of two consecutive tokens --merge--> new token
# add new token to vocab
# replace all consecutive two tokens to be merged to new token

In [5]:
# 遍历 当前 tokens_lst 以 计算 每个 pair of two consecutive tokens 出现的频数
def get_counts_of_pair(tokens_lst:list):
    counts = {}
    for pair in zip(tokens_lst, tokens_lst[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    
    return counts

stats = get_counts_of_pair(text_byteints)

In [ ]:
print(stats)
# list of following dict:
#   key: (token_a, token_b)
#   value: number
sort_stats = sorted( stats.items(), key=lambda x: x[1], reverse=True)
print(sort_stats)

{(239, 188): 1, (188, 181): 1, (181, 239): 1, (239, 189): 6, (189, 142): 1, (142, 239): 1, (189, 137): 1, (137, 239): 1, (189, 131): 1, (131, 239): 1, (189, 143): 1, (143, 239): 1, (189, 132): 1, (132, 239): 1, (189, 133): 1, (133, 33): 1, (33, 32): 2, (32, 240): 3, (240, 159): 15, (159, 133): 7, (133, 164): 1, (164, 240): 1, (133, 157): 1, (157, 240): 1, (133, 152): 1, (152, 240): 1, (133, 146): 1, (146, 240): 1, (133, 158): 1, (158, 240): 1, (133, 147): 1, (147, 240): 1, (133, 148): 1, (148, 226): 1, (226, 128): 12, (128, 189): 1, (189, 32): 1, (159, 135): 7, (135, 186): 1, (186, 226): 1, (128, 140): 6, (140, 240): 6, (135, 179): 1, (179, 226): 1, (135, 174): 1, (174, 226): 1, (135, 168): 1, (168, 226): 1, (135, 180): 1, (180, 226): 1, (135, 169): 1, (169, 226): 1, (135, 170): 1, (170, 33): 1, (159, 152): 1, (152, 132): 1, (132, 32): 1, (32, 84): 1, (84, 104): 1, (104, 101): 6, (101, 32): 20, (32, 118): 1, (118, 101): 3, (101, 114): 6, (114, 121): 2, (121, 32): 2, (32, 110): 2, (110,

In [11]:
# 测试 char-level
print( get_counts_of_pair(list('aaabdaaabac')) )

{('a', 'a'): 4, ('a', 'b'): 2, ('b', 'd'): 1, ('d', 'a'): 1, ('b', 'a'): 1, ('a', 'c'): 1}


In [12]:
# 遍历 当前 tokens_lst，把指定的 pair of two consecutive tokens 替换成 指定的 new token
# [5, 6, 6, 7, 9, 1] merge(6,6) 
def merge_pair(tokens_lst:list, pair, new_token):
    new_tokens_lst = []
    i = 0
    while i < len(tokens_lst):
        if i < len(tokens_lst) - 1 and tokens_lst[i] == pair[0] and tokens_lst[i+1] == pair[1]: # 当前 i, i+1 两个位置 组成 pair
            new_tokens_lst.append(new_token) # i 位置 记录 new_token
            i += 2 # i+1 位置被跳过, compress。下一轮 检查 i+2 位置
        else: # 当前 i, i+1 两个位置 不组成 pair, 或者 i 已经是 last token
            new_tokens_lst.append( tokens_lst[i] ) # i 位置 记录 原 i 位置 token
            i += 1 # 下一轮 检查 i+1位置
    
    return new_tokens_lst

In [18]:
# 基本功能测试
tokens_lst = [5, 6, 6, 7, 9, 1]
pair = (5, 6)
new_token = 99
print( merge_pair(tokens_lst, pair, new_token) )

[99, 6, 7, 9, 1]
